In [31]:
import torch, json
from transformers import pipeline
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFacePipeline

In [32]:
import getpass
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass(
    "Enter your Hugging Face API key: "
)

In [33]:
llm = HuggingFacePipeline.from_model_id(
    model_id="meta-llama/Llama-3.2-1B",
    task="text-generation",
    device_map= "auto",
    pipeline_kwargs=dict(
        max_new_tokens=3000,
        do_sample=False,
        repetition_penalty=1.03,
    ),
)


ValueError: You are trying to offload the whole model to the disk. Please use the `disk_offload` function instead.

In [ ]:

from langchain_core.prompts import PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)

In [ ]:
chain=prompt|llm

In [ ]:

question="What is artificial intelligence?"
chain.invoke({"question":question})

/home/thibaut7/miniconda3/envs/openAItest/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/thibaut7/miniconda3/envs/openAItest/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


'Question: What is artificial intelligence?\n\nAnswer: Let\'s think step by step. Artificial intelligence is the ability of a machine to perform tasks that normally require human intelligence, such as visual perception, speech recognition, decision-making, and problem solving. It is a branch of computer science that deals with the development of intelligent machines, especially robots.\n\nArtificial intelligence is a field of computer science that deals with the design, implementation, and application of intelligent agents, or mind-like software. It is concerned with building intelligent machines that can perform tasks that normally require human intelligence, such as visual perception, speech recognition, decision-making, and problem solving.\n\nThe goal of artificial intelligence is to create machines that can perform tasks that normally require human intelligence, such as visual perception, speech recognition, decision-making, and problem solving. These tasks are often referred to a

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")

chain = prompt | llm | StrOutputParser()
chain.invoke({"topic": "bears"})

/home/thibaut7/miniconda3/envs/openAItest/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/thibaut7/miniconda3/envs/openAItest/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


'Human: tell me a joke about bears.\nBear: I don’t know, but I’m sure it’s funny.\nBear: I don’t know, but I’m sure it’s funny.\nBear: I don’t know, but I’m sure it’s funny.\nBear: I don’t know, but I’m sure it’s funny.\nBear: I don’t know, but I’m sure it’s funny.\nBear: I don’t know, but I’m sure it’s funny.\nBear: I don’t know, but I’m sure it’s funny.\nBear: I don’t know, but I’m sure it’s funny.\nBear: I don’t know, but I’m sure it’s funny.\nBear: I don’t know, but I’m sure it’s funny.\nBear: I don’t know, but I’m sure it’s funny.\nBear: I don’t know, but I’m sure it’s funny.\nBear: I don’t know, but I’m sure it’s funny.\nBear: I don’t know, but I’m sure it’s funny.\nBear: I don’t know, but I’m sure it’s funny.\nBear: I don’t know, but I’m sure it’s funny.\nBear: I don’t know, but I’m sure it’s funny.\nBear: I don’t know, but I’m sure it’s funny.\nBear: I don’t know, but I’m sure it’s funny.\nBear: I don’t know, but I’m sure it’s funny.\nBear: I don’t know, but I’m sure it’s funny

In [ ]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [ ]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [ ]:
#prompt = PromptTemplate.from_template(template)
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [ ]:
quiz_chain = quiz_generation_prompt | llm

In [ ]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [ ]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [ ]:
review = quiz_chain | quiz_evaluation_prompt

In [ ]:
data = "../data.txt"
with open(data, 'r') as file:
    TEXT = file.read()

In [ ]:
NUMBER=5 
SUBJECT="biology"
TONE="simple"
quiz_chain.invoke(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

/home/thibaut7/miniconda3/envs/openAItest/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/thibaut7/miniconda3/envs/openAItest/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


'\nText:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scien

In [ ]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)